In [ ]:
 # Aldo Andika

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df_male = pd.read_csv('/content/drive/MyDrive/Final Project/dataset/application_history_m.csv', sep = ';')
df_female = pd.read_csv('/content/drive/MyDrive/Final Project/dataset/application_history_f.csv', sep = ';')
df_credit = pd.read_csv('/content/drive/MyDrive/Final Project/dataset/credit_history.csv')

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
# Untuk Balance Data
from imblearn.under_sampling import RandomUnderSampler

# Decision tree, XGBoost 
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline

# Metrics 

build a classifier model to help Bank Talenta Rakyat to gauge their customers efficiently and effectively also optimize the process of screening customer profiles that apply for credit that indicate the chances of borrowers defaulting (cannot repay their loan in more than 59 days)

# Data Cleaning / Data Preparation

## Perempuan

In [ ]:
df_female.head()

,Id_customer,JK,KepemilikanMobil,KepemilikanProperti,JmlAnak,Pendapatan,TipePendapatan,TingkatPendidikan,StatusKeluarga,TipeRumah,FlagMobile,FlagWorkPhone,FlagPhone,Email,Pekerjaan,JmlAnggotaKeluarga,Age,Experience
0,5008808,Perempuan,Tidak,Ya,0,270000,Asosiasi komersial,G,NM,MH,1,0,1,1,Sales staff,1,52,8
1,5008809,Perempuan,Tidak,Ya,0,270000,Asosiasi komersial,G,NM,MH,1,0,1,1,Sales staff,1,52,8
2,5008810,Perempuan,Tidak,Ya,0,270000,Asosiasi komersial,G,NM,MH,1,0,1,1,Sales staff,1,52,8
3,5008811,Perempuan,Tidak,Ya,0,270000,Asosiasi komersial,G,NM,MH,1,0,1,1,Sales staff,1,52,8
4,5008812,Perempuan,Tidak,Ya,0,283500,Pensioner,PG,D,MH,1,0,0,0,NaN,1,62,1001


In [ ]:
print(df_female.shape)

(294360, 18)


## Laki-laki

In [ ]:
df_male.head()

,Id_customer,JK,KepemilikanMobil,KepemilikanProperti,JmlAnak,Pendapatan,TipePendapatan,TingkatPendidikan,StatusKeluarga,TipeRumah,FlagMobile,FlagWorkPhone,FlagPhone,Email,Pekerjaan,JmlAnggotaKeluarga,Age,Experience
0,5008804,Laki-laki,Ya,Ya,0,427500,Bekerja,PG,M,RA,1,1,0,0,NaN,2,33,12
1,5008805,Laki-laki,Ya,Ya,0,427500,Bekerja,PG,M,RA,1,1,0,0,NaN,2,33,12
2,5008806,Laki-laki,Ya,Ya,0,112500,Bekerja,G,M,MH,1,0,0,0,Security staff,2,59,3
3,5008815,Laki-laki,Ya,Ya,0,270000,Bekerja,PG,M,MH,1,1,1,1,Accountants,2,46,2
4,5112956,Laki-laki,Ya,Ya,0,270000,Bekerja,PG,M,MH,1,1,1,1,Accountants,2,46,2


In [ ]:
print(df_male.shape)

(144096, 18)


## Credit History Data

In [ ]:
df_credit.head()

,Id_customer,Overdue
0,5001711,Tidak memiliki pinjaman
1,5001711,0
2,5001711,0
3,5001711,0
4,5001712,0


In [ ]:
df_credit['Overdue'].value_counts()

0                          821230
Tidak memiliki pinjaman    208755
1                           10936
5                            1677
2                             849
3                             317
4                             222
Name: Overdue, dtype: int64

In [ ]:
# Ganti value tidak memiliki pinjaman jadi 0
df_credit['Overdue'] = df_credit['Overdue'].replace(['Tidak memiliki pinjaman'], '0')

# Ambil maksimum overdue dari setiap Borrowers
g = df_credit.groupby(['Id_customer'])['Overdue'].transform('max')
df_credit = df_credit[(df_credit['Overdue'] == g)]

df_credit.head()

,Id_customer,Overdue
0,5001711,0
1,5001711,0
2,5001711,0
3,5001711,0
4,5001712,0


In [ ]:
# Ubah value overdue
df_credit['Overdue'] = np.where(df_credit['Overdue']=='0', 0, 1)

In [ ]:
df_credit['Overdue'].value_counts()

0    902030
1     11504
Name: Overdue, dtype: int64

## Combine 3 file

1. Borrowers dikatakan memiliki badscore jika tidak bisa membayar pinjaman diatas 30 hari.
2. Untuk Credit history data, ambil maksimum overdue dari setiap Borrowers.
3. Nasabah yang digunakan untuk membuat model hanya yang memiliki history pinjaman(overdue) ---tidak memiliki pinjaman- diklasifikasikan jadi 0 1--

In [ ]:
combine = pd.concat([df_male, df_female])

combine.head()

,Id_customer,JK,KepemilikanMobil,KepemilikanProperti,JmlAnak,Pendapatan,TipePendapatan,TingkatPendidikan,StatusKeluarga,TipeRumah,FlagMobile,FlagWorkPhone,FlagPhone,Email,Pekerjaan,JmlAnggotaKeluarga,Age,Experience
0,5008804,Laki-laki,Ya,Ya,0,427500,Bekerja,PG,M,RA,1,1,0,0,NaN,2,33,12
1,5008805,Laki-laki,Ya,Ya,0,427500,Bekerja,PG,M,RA,1,1,0,0,NaN,2,33,12
2,5008806,Laki-laki,Ya,Ya,0,112500,Bekerja,G,M,MH,1,0,0,0,Security staff,2,59,3
3,5008815,Laki-laki,Ya,Ya,0,270000,Bekerja,PG,M,MH,1,1,1,1,Accountants,2,46,2
4,5112956,Laki-laki,Ya,Ya,0,270000,Bekerja,PG,M,MH,1,1,1,1,Accountants,2,46,2


In [ ]:
combine['JK'].value_counts()

Perempuan    294360
Laki-laki    144096
Name: JK, dtype: int64

In [ ]:
combine.shape

(438456, 18)

In [ ]:
# Gabung Tabel

df = pd.merge(combine, df_credit, on=['Id_customer'], how='inner')

In [ ]:
df.head()

,Id_customer,JK,KepemilikanMobil,KepemilikanProperti,JmlAnak,Pendapatan,TipePendapatan,TingkatPendidikan,StatusKeluarga,TipeRumah,FlagMobile,FlagWorkPhone,FlagPhone,Email,Pekerjaan,JmlAnggotaKeluarga,Age,Experience,Overdue
0,5008804,Laki-laki,Ya,Ya,0,427500,Bekerja,PG,M,RA,1,1,0,0,NaN,2,33,12,1
1,5008805,Laki-laki,Ya,Ya,0,427500,Bekerja,PG,M,RA,1,1,0,0,NaN,2,33,12,1
2,5008805,Laki-laki,Ya,Ya,0,427500,Bekerja,PG,M,RA,1,1,0,0,NaN,2,33,12,1
3,5008806,Laki-laki,Ya,Ya,0,112500,Bekerja,G,M,MH,1,0,0,0,Security staff,2,59,3,0
4,5008806,Laki-laki,Ya,Ya,0,112500,Bekerja,G,M,MH,1,0,0,0,Security staff,2,59,3,0


In [ ]:
df.isna().sum()

Id_customer                 0
JK                          0
KepemilikanMobil            0
KepemilikanProperti         0
JmlAnak                     0
Pendapatan                  0
TipePendapatan              0
TingkatPendidikan           0
StatusKeluarga              0
TipeRumah                   0
FlagMobile                  0
FlagWorkPhone               0
FlagPhone                   0
Email                       0
Pekerjaan              212213
JmlAnggotaKeluarga          0
Age                         0
Experience                  0
Overdue                     0
dtype: int64

In [ ]:
df.isnull().mean() * 100

Id_customer             0.000000
JK                      0.000000
KepemilikanMobil        0.000000
KepemilikanProperti     0.000000
JmlAnak                 0.000000
Pendapatan              0.000000
TipePendapatan          0.000000
TingkatPendidikan       0.000000
StatusKeluarga          0.000000
TipeRumah               0.000000
FlagMobile              0.000000
FlagWorkPhone           0.000000
FlagPhone               0.000000
Email                   0.000000
Pekerjaan              31.375561
JmlAnggotaKeluarga      0.000000
Age                     0.000000
Experience              0.000000
Overdue                 0.000000
dtype: float64

In [ ]:
df[df.isna().any(axis=1)]

,Id_customer,JK,KepemilikanMobil,KepemilikanProperti,JmlAnak,Pendapatan,TipePendapatan,TingkatPendidikan,StatusKeluarga,TipeRumah,FlagMobile,FlagWorkPhone,FlagPhone,Email,Pekerjaan,JmlAnggotaKeluarga,Age,Experience,Overdue
0,5008804,Laki-laki,Ya,Ya,0,427500,Bekerja,PG,M,RA,1,1,0,0,NaN,2,33,12,1
1,5008805,Laki-laki,Ya,Ya,0,427500,Bekerja,PG,M,RA,1,1,0,0,NaN,2,33,12,1
2,5008805,Laki-laki,Ya,Ya,0,427500,Bekerja,PG,M,RA,1,1,0,0,NaN,2,33,12,1
491,5008951,Laki-laki,Tidak,Ya,1,225000,Bekerja,G,M,MH,1,0,0,0,NaN,3,28,8,0
492,5008951,Laki-laki,Tidak,Ya,1,225000,Bekerja,G,M,MH,1,0,0,0,NaN,3,28,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676341,5148679,Perempuan,Tidak,Tidak,0,121500,Asosiasi komersial,G,M,MH,1,0,0,0,NaN,2,52,17,1
676345,5149055,Perempuan,Tidak,Ya,0,112500,Asosiasi komersial,G,M,MH,1,1,1,0,NaN,2,43,7,1
676346,5149055,Perempuan,Tidak,Ya,0,112500,Asosiasi komersial,G,M,MH,1,1,1,0,NaN,2,43,7,1
676347,5149056,Perempuan,Tidak,Ya,0,112500,Asosiasi komersial,G,M,MH,1,1,1,0,NaN,2,43,7,1


In [ ]:
df.shape

(676364, 19)

In [ ]:
# Drop NaN

df = df.dropna(subset=['Pekerjaan'], inplace=False)

In [ ]:
df.duplicated().sum()

439158

In [ ]:
# Drop data duplikat

data = df.drop_duplicates(keep='last')

In [ ]:
# Dataset untuk diolah di Tableau
data.to_csv('df_tableau.csv', header=True, index=False)

In [ ]:
data['Overdue'].value_counts()

0    21958
1     3035
Name: Overdue, dtype: int64